In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
df = pd.read_excel("https://github.com/Xanitross/Covid_Effects_On_Jobs/blob/main/All_Employees_Thousands_NonFarm.xlsx?raw=true", skiprows=12)
df

c:\Users\Xanit\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2013,135265,135544,135689,135876,136091,136269,136391.0,136635.0,136819.0,137039.0,137314.0,137368.0
1,2014,137552,137707,137986,138298,138501,138833,139076.0,139258.0,139563.0,139805.0,140089.0,140366.0
2,2015,140562,140829,140924,141202,141540,141695,141989.0,142130.0,142265.0,142585.0,142810.0,143083.0
3,2016,143198,143410,143666,143856,143901,144152,144515.0,144664.0,144961.0,145069.0,145189.0,145408.0
4,2017,145639,145845,145975,146172,146389,146588,146772.0,146907.0,146999.0,147147.0,147377.0,147521.0
5,2018,147670,148058,148281,148426,148755,148966,149022.0,149272.0,149361.0,149526.0,149622.0,149805.0
6,2019,150114,150092,150320,150563,150630,150797,150879.0,151111.0,151318.0,151447.0,151662.0,151764.0
7,2020,152098,152371,150944,130430,133055,137620,139064.0,140799.0,141760.0,142479.0,142743.0,142475.0
8,2021,142969,143544,144328,144614,145096,145789,146558.0,147221.0,147778.0,148559.0,149173.0,149742.0
9,2022,150106,151010,151424,151678,152042,152412,152980.0,153332.0,153682.0,154006.0,154296.0,154535.0


In [20]:
df = pd.read_excel("https://github.com/Xanitross/Covid_Effects_On_Jobs/blob/main/Unemployment_Rate_Monthly.xlsx?raw=true", skiprows=11)
df

c:\Users\Xanit\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2013,8.0,7.7,7.5,7.6,7.5,7.5,7.3,7.2,7.2,7.2,6.9,6.7
1,2014,6.6,6.7,6.7,6.2,6.3,6.1,6.2,6.1,5.9,5.7,5.8,5.6
2,2015,5.7,5.5,5.4,5.4,5.6,5.3,5.2,5.1,5.0,5.0,5.1,5.0
3,2016,4.8,4.9,5.0,5.1,4.8,4.9,4.8,4.9,5.0,4.9,4.7,4.7
4,2017,4.7,4.6,4.4,4.4,4.4,4.3,4.3,4.4,4.3,4.2,4.2,4.1
5,2018,4.0,4.1,4.0,4.0,3.8,4.0,3.8,3.8,3.7,3.8,3.8,3.9
6,2019,4.0,3.8,3.8,3.6,3.7,3.6,3.7,3.7,3.5,3.6,3.6,3.6
7,2020,3.5,3.5,4.4,14.7,13.2,11.0,10.2,8.4,7.9,6.9,6.7,6.7
8,2021,6.3,6.2,6.1,6.1,5.8,5.9,5.4,5.2,4.8,4.5,4.2,3.9
9,2022,4.0,3.8,3.6,3.6,3.6,3.6,3.5,3.7,3.5,3.7,3.6,3.5


In [21]:
df = pd.read_csv("https://github.com/Xanitross/Covid_Effects_On_Jobs/blob/main/data_table_for_weekly_covid19_hospital_admissions_-_the_united_states.csv?raw=true", skiprows=1)
df

Date generated: Tue Jul 18 2023 15:18:58 GMT-0400 (Eastern Daylight Time)
Geography     Date                       Weekly COVID-19 Hospital Admissions                       
United States Jul  8 2023                                               6228                       
              Jul  1 2023                                               6282                       
              Jun 24 2023                                               6277                       
              Jun 17 2023                                               6612                       
...                                                                      ...                       
              Feb  8 2020                                                NaN                       
              Feb  1 2020                                                NaN                       
              Jan 25 2020                                                NaN                       
              Jan 18 2020                                                NaN                       
              Jan 11 2020                                                NaN                       

[184 rows x 1 columns]

In [22]:
df = pd.read_csv("https://github.com/Xanitross/Covid_Effects_On_Jobs/blob/main/data_table_for_weekly_deaths__the_united_states.csv?raw=true", skiprows=2)
df

,Geography,Date,Weekly Deaths,Death Data As Of
0,United States,Jul 8 2023,106,Jul 12 2023
1,United States,Jul 1 2023,338,Jul 12 2023
2,United States,Jun 24 2023,468,Jul 12 2023
3,United States,Jun 17 2023,550,Jul 12 2023
4,United States,Jun 10 2023,576,Jul 12 2023
...,...,...,...,...
178,United States,Feb 8 2020,4,Jul 12 2023
179,United States,Feb 1 2020,0,Jul 12 2023
180,United States,Jan 25 2020,3,Jul 12 2023
181,United States,Jan 18 2020,2,Jul 12 2023
